In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline


import argparse
import asyncio
import dataclasses
import glob
import logging
import multiprocessing
import os
import pathlib
import pprint
import sys
from typing import Any

import polars as pl
from omegaconf import OmegaConf, DictConfig
from tqdm import tqdm

from eval import kratos_utils
from eval.aggregation import eval_aggregation
from eval.data import EvaluationResultContainer
from eval.schema import EvalConfig

import logging
import os
from dataclasses import dataclass
from typing import Optional

logger = logging.getLogger("alpasim_eval_dev_notebook")
from eval.main import *
import numpy as np
import shapely
from rich import print
import rich


In [2]:
args = {
    "asl_search_glob": "/home/migl/workspace/alpasim/.wizard/asl/clipgt-d8cbf4ca-b7ff-44bd-a5be-260f736a02fe/15f2c488-10ad-11f0-b123-0242c0a84004/**/*.asl",
    # "config_path": "/home/migl/workspace/alpasim/.wizard/eval-config.yaml",
    "config_path": "/home/migl/workspace/alpasim/src/eval/example_config.yaml",
    "output_dir": "/home/migl/workspace/alpasim/.wizard/eval",
}

args = DictConfig(args)

config_untyped = OmegaConf.load(args.config_path)
cfg: EvalConfig = OmegaConf.merge(EvalConfig, config_untyped)
cfg.database.upload = False

In [ ]:
# Create output directory
os.makedirs(args.output_dir, exist_ok=True)

logger.info("Config details:")
logger.info(pprint.pformat(OmegaConf.to_container(cfg, resolve=True)))

# Option 2: Use the current event loop
# evaluation_result_containers = asyncio.run(get_evaluation_result_containers(args, cfg))
evaluation_result_containers = await get_evaluation_result_containers(args, cfg)

evaluation_result_containers = get_metric_results(evaluation_result_containers, cfg)

run_metadata = kratos_utils.get_metadata(pathlib.Path(args.config_path))
# Contains: timestamp_us, value, valid, metric_name, clipgt_id, batch_id, rollout_id
df = pl.concat(
    [
        df_from_evaluation_result_container(eval_result_container, run_metadata)
        for eval_result_container in evaluation_result_containers
    ]
)

logger.info(f"Writing metrics df to {args.output_dir}/metrics.parquet")
df.write_parquet(os.path.join(args.output_dir, "metrics.parquet"))
processed_dfs = eval_aggregation.aggregate_and_write_metrics_results_txt(df)

In [ ]:
print(filtered_timestamps[0] / 1e6, filtered_timestamps[-1] / 1e6)

In [ ]:
t1 = full_gt_trajectory
t2 = full_gt_trajectory.interpolate_to_timestamps(
    simulation_result.timestamps_us[ts_idx : ts_idx + 1]
)
# print(t1.time_range_us.stop)
# print(t2.time_range_us.stop)
print(t1.poses.vec3[-1])
print(t2.poses.vec3[-1])

print(list(full_gt_linestring.coords)[-1])
print(list(current_gt_linestring.coords)[-1])

In [ ]:
current_gt_linestring.length

In [ ]:
full_gt_linestring.length

In [ ]:
ego_

In [ ]:
ts

In [ ]:
from eval.scorers.wrong_lane import *

wrong_lane_scorer = WrongLaneScorer(cfg)
wrong_lane_scorer.debug_plot(simulation_result, ts, "curr_lanes")
wrong_lane_scorer.debug_plot(simulation_result, ts, "possible_current_lanes")

In [ ]:
%matplotlib inline




In [ ]:
fig, ax = plt.subplots()
for lane_polygon in curr_lanes:
    shapely_plotting.plot_polygon(get_lane_polygon(lane_polygon), ax=ax)
shapely_plotting.plot_polygon(ego_polygon, ax=ax)


# fig.show()

In [ ]:
curr_lanes[0].center

In [ ]:
curr_lane_idx = np.where(ego_in_lane)[0][0]


lane_polygon = get_lane_polygon(lane)
print(lane_polygon.contains(ego_polygon.centroid))

In [ ]:
shapely.LineString(lane.center.points[..., :2]).buffer(1.8)